In [2]:
import requests as req
from bs4 import BeautifulSoup as bs
import requests as req
import pandas as pd

# Web Scrapping (ESPN)

In [3]:
url = 'https://www.espn.com/soccer/standings/_/league/ESP.1/season/2019'

In [4]:
html=req.get(url).content

In [5]:
soup=bs(html, 'html.parser')

In [6]:
table_html = soup.find_all('div', class_="flex")

In [8]:
index = []

for e in table_html:
    if e.tr:
        index.append(e.tr.text)

In [16]:
index  # Realmente extraer esto no nos aporta demasiado, pero nos permite comprender mejor la estructura html de esta url.

['2019-2020']

In [10]:
# El nombre de la primera columna (index) tiene diferente etiquetado html que el resto de nombres de columnas (tienen un div de otra clase).
# Tenemos que acceder al html del resto de las columnas ('div', class_="Table__Scroller").

table_html_2 = soup.find_all('div', class_="Table__Scroller")

In [11]:
# Ya tenemos el div completo. Ahra tenemos que acceder al nombre de las columnas (texto de <a> dentro del <div>).

col_html = []
for e in table_html_2:
    col_html.append(e.find_all('a'))
       

In [12]:
col_html  # Ya tenemos los diferentes tags <a>.

[[<a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/gamesplayed/dir/desc/season/2019" tabindex="0">GP</a>,
  <a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/wins/dir/desc/season/2019" tabindex="0">W</a>,
  <a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/ties/dir/desc/season/2019" tabindex="0">D</a>,
  <a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/losses/dir/asc/season/2019" tabindex="0">L</a>,
  <a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/pointsfor/dir/desc/season/2019" tabindex="0">F</a>,
  <a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/pointsagainst/dir/asc/season/2019" tabindex="0">A</a>,
  <a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/pointdifferential/dir/desc/season/2019" tabindex="0">GD</a>,
  <a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/points/dir/desc/season/2019" tabindex="0">P</a>]]

In [14]:
# Accedemos al texto (doble bucle porque 'row_html' es una lista dentro de otra lista (hicimos find_all)).

col_names=[]
for e in col_html:
    for i in e:
        col_names.append(i.text)

In [15]:
col_names # Ya tenemos los nombres de las columnas (primer paso para ir construyendo nuestro DataFrame)

['GP', 'W', 'D', 'L', 'F', 'A', 'GD', 'P']

Ahora que ya tenemos el nombre de las columnas, seguimos extrayendo los datos.

Como la primera columna (equipos) está estructurada de manera independiente a las otras, debemos hacerlo por separado (primero extraeremos los equipos y luego los valores de la tabla).

In [17]:
# Empezamos extrayendo los nombres de los equipos.

teams_html = soup.find_all('span', class_="hide-mobile")

In [18]:
teams = []
for e in teams_html:
    teams.append(e.text)

In [19]:
teams # Ya tenemos los nombres de los equipos (primera columna)

['Real Madrid',
 'Barcelona',
 'Atletico Madrid',
 'Sevilla',
 'Villarreal',
 'Real Sociedad',
 'Granada',
 'Getafe',
 'Valencia',
 'Osasuna',
 'Athletic Club',
 'Levante',
 'Real Valladolid',
 'Eibar',
 'Real Betis',
 'Alavés',
 'Celta Vigo',
 'Leganés',
 'Mallorca',
 'Espanyol']

Ahora que tenemos los nombres de las columnas y los equipos, extraemos los datos de la tabla.

In [20]:
data_html = soup.find_all('tbody', class_= "Table__TBODY")

In [21]:
len(data_html)

2

In [22]:
# Seleccionamos los 'tr' en el segundo elemento de data_html (es el 'tbody' de la clase "Table__TBODY" que nos interesa).

data_html2 = data_html[1].find_all('tr')

In [25]:
# Primero hacemos la prueba para un elemento de data_html2.

data_html2[1]

<tr class="filled Table__TR Table__TR--sm Table__even" data-idx="1"><td class="Table__TD"><span class="stat-cell">38</span></td><td class="Table__TD"><span class="stat-cell">25</span></td><td class="Table__TD"><span class="stat-cell">7</span></td><td class="Table__TD"><span class="stat-cell">6</span></td><td class="Table__TD"><span class="stat-cell">86</span></td><td class="Table__TD"><span class="stat-cell">38</span></td><td class="Table__TD"><span class="stat-cell clr-positive">+48</span></td><td class="Table__TD"><span class="stat-cell">82</span></td></tr>

In [26]:
data_html2[1].find_all('td')

[<td class="Table__TD"><span class="stat-cell">38</span></td>,
 <td class="Table__TD"><span class="stat-cell">25</span></td>,
 <td class="Table__TD"><span class="stat-cell">7</span></td>,
 <td class="Table__TD"><span class="stat-cell">6</span></td>,
 <td class="Table__TD"><span class="stat-cell">86</span></td>,
 <td class="Table__TD"><span class="stat-cell">38</span></td>,
 <td class="Table__TD"><span class="stat-cell clr-positive">+48</span></td>,
 <td class="Table__TD"><span class="stat-cell">82</span></td>]

In [29]:
[e.text for e in data_html2[1].find_all('td')]  

# Comprobamos que funciona, así que aplicamos el proceso anterior a todo el contenido de la tabla.

['38', '25', '7', '6', '86', '38', '+48', '82']

In [30]:
# Primero creamos una lista con todos los 'td' en todos los elemento del 'tbody'.

data_prov = []
for e in data_html2:
    data_prov.append(e.find_all('td'))

In [31]:
# Aplicamos el doble bucle sobre la lista previamente creada.

# El doble bucle con la lista 'prov' nos permite añadir a data_def listas con los valores de cada fila.
# Es decir, data_def será una lista de listas (podemos meterlo como data en un DataFrame).
#Si no hacemos esto, lo que tendríamos es una única lista con todos los valores y esto no es lo que queremos.

data_def = []

for e in data_prov:
    prov = []
    
    for i in e:
        prov.append(i.text)
        
    data_def.append(prov)

In [32]:
# Observamos nuestro data (cada elemento de la lista, es una lista que contiene los valores para una fila).

data_def

[['38', '26', '9', '3', '70', '25', '+45', '87'],
 ['38', '25', '7', '6', '86', '38', '+48', '82'],
 ['38', '18', '16', '4', '51', '27', '+24', '70'],
 ['38', '19', '13', '6', '54', '34', '+20', '70'],
 ['38', '18', '6', '14', '63', '49', '+14', '60'],
 ['38', '16', '8', '14', '56', '48', '+8', '56'],
 ['38', '16', '8', '14', '52', '45', '+7', '56'],
 ['38', '14', '12', '12', '43', '37', '+6', '54'],
 ['38', '14', '11', '13', '46', '53', '-7', '53'],
 ['38', '13', '13', '12', '46', '54', '-8', '52'],
 ['38', '13', '12', '13', '41', '38', '+3', '51'],
 ['38', '14', '7', '17', '47', '53', '-6', '49'],
 ['38', '9', '15', '14', '32', '43', '-11', '42'],
 ['38', '11', '9', '18', '39', '56', '-17', '42'],
 ['38', '10', '11', '17', '48', '60', '-12', '41'],
 ['38', '10', '9', '19', '34', '59', '-25', '39'],
 ['38', '7', '16', '15', '37', '49', '-12', '37'],
 ['38', '8', '12', '18', '30', '51', '-21', '36'],
 ['38', '9', '6', '23', '40', '65', '-25', '33'],
 ['38', '5', '10', '23', '27', '58',

Generamos el DataFrame a partir del scrapeo de ESPN.

In [33]:
col_names

['GP', 'W', 'D', 'L', 'F', 'A', 'GD', 'P']

In [34]:
teams # Será el index de nuestro DataFrame

['Real Madrid',
 'Barcelona',
 'Atletico Madrid',
 'Sevilla',
 'Villarreal',
 'Real Sociedad',
 'Granada',
 'Getafe',
 'Valencia',
 'Osasuna',
 'Athletic Club',
 'Levante',
 'Real Valladolid',
 'Eibar',
 'Real Betis',
 'Alavés',
 'Celta Vigo',
 'Leganés',
 'Mallorca',
 'Espanyol']

In [35]:
len(col_names)==len(data_def[0])   # Podemos generar el DataFrame sin problemas.

True

In [36]:
len(teams)==len(data_def)     # Sí coincide así que no ha habido fallos (hay tantos equipos como registros de valores (filas)).

True

In [41]:
teams_stats = pd.DataFrame(data_def, columns = col_names)

In [42]:
teams_stats  # Tenemos el DataFrame con todos los valores ('data_def'). Ahora añadiremos los equipos y lo definiremos como index.

,GP,W,D,L,F,A,GD,P
0,38,26,9,3,70,25,+45,87
1,38,25,7,6,86,38,+48,82
2,38,18,16,4,51,27,+24,70
3,38,19,13,6,54,34,+20,70
4,38,18,6,14,63,49,+14,60
5,38,16,8,14,56,48,+8,56
6,38,16,8,14,52,45,+7,56
7,38,14,12,12,43,37,+6,54
8,38,14,11,13,46,53,-7,53
9,38,13,13,12,46,54,-8,52


In [45]:
# Añadimos la columna 'teams'.

teams_stats['Teams'] = teams

In [46]:
# Definimos 'Team' como index.

teams_stats.set_index('Teams',inplace = True)

In [48]:
# Ya tenemos nuestro DataFrame perfectamente estructurado.
# Debemos tener en cuenta que los valores recogidos en el DF son de tipo string.
# Esto ahora no supone ningún problema, pero es algo que debemos tener en cuenta si queremos trabajar con los datos más adelante.

teams_stats

,GP,W,D,L,F,A,GD,P
Teams,,,,,,,,
Real Madrid,38,26,9,3,70,25,+45,87
Barcelona,38,25,7,6,86,38,+48,82
Atletico Madrid,38,18,16,4,51,27,+24,70
Sevilla,38,19,13,6,54,34,+20,70
Villarreal,38,18,6,14,63,49,+14,60
Real Sociedad,38,16,8,14,56,48,+8,56
Granada,38,16,8,14,52,45,+7,56
Getafe,38,14,12,12,43,37,+6,54
Valencia,38,14,11,13,46,53,-7,53
